# Анализ рынка недвижимости в Санкт-Петербурге

**Цель:** Определить рыночную стоимость объектов недвижимости и типичные параметры квартир, которые влияют на стоимость

**Описание:** На основе архива данных провести исследование объявлений о продаже квартир. Результатом исследования должны стать выводы о рыночной стоимости недвижимости. Для этого необходимо отсечь аномальные значения и проверить предоставленные данные.

---
Для исследования использовал данные сервисов Яндекс.Недвижимость.

---
В работе использовал библиотеки pandas, matplotlib, seaborn

### Содержание
[1. Загрузка данных](#загрузка-данных)

[2. Предобработка данных](#предобработка-данных)
- [Пропуски в столбцах](#пропуски)
- [Изменим тип данных](#изменим-тип-данных)
- [Аномальные значения и выбросы](#аномальные-значения-и-выбросы)
- [Неявные дубликаты](#неявные-дубликаты)

[3. Расчёты и добавление результатов в таблицу](#расчёты)

[4. Исследовательский анализ данных](#исследовательский-анализ-данных)
- [Изучим параметры квартир](#изучим-параметры-квартир)
- [Выводы по гистограммам](#выводы-по-гистограммам)
- [Изучим время продажи квартир](#изучим-время-продажи)
- [Какие факторы влияют на цену квартиры](#факторы-влияют-на-цену)
- [10 населённых пунктов с наибольшим числом объявлений](#10-населённых-пунктов)
- [Предложения квартира в центрально части Санкт-Петербурга](#центр-санкт-петербург)

[5. Общие выводы по проекту](#общий-вывод)

## <a id="загрузка-данных">1. Загрузка данных из файла</a>

In [ ]:
# подключаем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime

In [ ]:
# загружаем данные
try:
    data = pd.read_csv('real_estate_data.csv')
except:
    data = pd.read_csv('/datasets/real_estate_data.csv')

In [ ]:
# посмотим на данные
data.info()
data.head()

Видим, что данные склеились и их невозможно оценить. Разделим данные.

In [ ]:
#Разделим данные из файла на столбцы
#Разделить в данном случае "\t"
try:
    data = pd.read_csv('real_estate_data.csv', sep='\t')
except:
    data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')

In [ ]:
# выведем данные
print(data.head())
print(data.info())

In [ ]:
#Скопируем исходный датасет, чтоб иметь возможность сравнить с тем, что получится после фильтрации
data_raw = data.copy()

**Промежуточные итоги:**
- Разделили "склеенные" столбцы
- В данных имеются пропуски
- В некоторых столбцах не подходящий тип данных

## <a id="предобработка-данных">2. Предобработка данных</a>

<a id="пропуски">**Пропуски в столбцах**</a>

**ceiling_height** - высота потолков. Скорее всего она зависит от количества этажей в доме floors_total. В столбце floors_total тоже имеются пропуски. Поэтому сначала заполним пропуски в floors_total и потом в ceiling_height.

**floors_total** общее число этаже в доме скорее всего типично для одного населенного пункта. Заполним пропуски медианным значениями в зависимости от населённого пункта из столбца locality_name.

В столбце **locality_name** есть пропуски. Для задачи нам важно где продаётся квартира, поэтому эти пропуски нужно либо заполнить либо убрать, чтоб анализ был верным. Так как эти пропуски невозможно заполнить, удалим строки с этими пропусками.

In [ ]:
# Удалим строки с пропускми в столбце locality_name

data = data.dropna(subset=['locality_name'])

In [ ]:
# Заполним пропуски в floors_total медианным значением в зависимости от населённого пункта
# В цикле перебираем названием населённых пунктов и берём медианно значение количества этажей

for locality_name in data['locality_name'].unique():
    data.loc[data['locality_name'] == locality_name, 'floors_total'] = data.loc[data['locality_name'] == locality_name, 'floors_total'].fillna(data['floors_total'].median())

In [ ]:
# Заполним пропуски в ceiling_height в зависимости от количества этажей медианным значением
# В цикле перебираем количество этажей и присваем медианное значение для высоты потолков

for floors_total in data['floors_total'].unique():
    data.loc[data['floors_total'] == floors_total, 'ceiling_height'] = data.loc[data['floors_total'] == floors_total, 'ceiling_height'].fillna(data['ceiling_height'].median())

Пропуски в столбце **living_area**

Может зависеть от количества комнат в квартире столбец rooms. Заполним пропуски в этом столбце

In [ ]:
# Заполним пропуски в living_area мидеанным значениями в зависимости от количества комнат rooms

for rooms in data['rooms'].unique():
    data.loc[data['rooms'] == rooms, 'living_area'] = data.loc[data['rooms'] == rooms, 'living_area'].fillna(data['living_area'].median())

В столбце **is_apartment**

Очень много пропусков. Столбец предполагает значения булевого типа. Возможно пользователи не заполняли его, т.е. если "да" - то заполняли, если "нет" - оставляли пустым. Поэтому заполним пропуски значением "нет".

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(False)

Пропуски в столбце **kitchen_area**

Возможно связаны с тем, что квартира-студия и нет отдельной кухни. Поэтому не указана площадь кухни. Посчитаем количество студия и сравним с колиеством пропусков

In [ ]:
# посчитаем количество студий

data['studio'].value_counts()

Количество студий 149, а пропусков kitchen_area больше

Сначала проверим указана площадь кухни для студий

In [ ]:
# Посмотрим площадь кухни для студий

data[data['studio'] == True]['kitchen_area'].unique()

Для студии площадь кухни не указана, значит эти пропуски можно заменить на "0"

А для квартир не студий заполним пропуски медианным значением

In [ ]:
#заполним пропуски для студий

data.loc[data['studio'] == True,'kitchen_area'] = data.loc[data['studio'] == True,'kitchen_area'].fillna(0)

In [ ]:
#заполним пропуски для остальных квартир

data.loc[data['studio'] == False,'kitchen_area'] = data.loc[data['studio'] == False,'kitchen_area'].fillna(data['kitchen_area'].median())

Пропуски в столбце **balcony**

Скорее всего связаны с тем, что при отсутсвии балкона поле не заполняли

Поэтому заполним пропуски значеним "0"

In [ ]:
# Заполним пропуски в столбце balcony
data['balcony'] = data['balcony'].fillna(0)

Пропуски в столбце **airports_nearest**

Можно заполнить медианным значением, учитвая в каком населённом пункте находится квартира

In [ ]:
# Заполним расстояние до ближайшего аэропорта медианными значениями, учитывая населённый пункт

for locality_name in data['locality_name'].unique():
    data.loc[data['locality_name'] == locality_name, 'airports_nearest'] = data.loc[data['locality_name'] == locality_name, 'airports_nearest'].fillna(data['airports_nearest'].median())

Пропуски в **cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest**

Вероятно эти пропуски связанны с тем, что далеко не все населённые пункты являются крупными и нет понятия "центр города". Возможно поблизости нет парков и водоёмов. Эти значения невозможно заменить логически, поэтому в пропуски поставим значение "0".

In [ ]:
# Заменим пропуски в столбцах на значение "0"
# Создадим список и в цикле заменим значения

for index in {'cityCenters_nearest', 'parks_around3000', 'parks_nearest', 'ponds_around3000', 'ponds_nearest'}:
    data[index] = data[index].fillna(0)

Пропуски в **days_exposition**

Количество дней показа до момента снятия объявления. Вероятно пропуски связанны с тем, что объявление не было снято на момент выгрузки данных в файл. Поэтому заполним пропуски как разницу между текущей датой и первым днём публикации.

In [ ]:
# Переведём столбец first_day_exposition в формат даты

data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%d %H:%M')

In [ ]:
#Напишем функцию для Заполния пропусков значением "текущаяя дата - дата публикции"

def days_exposition(wtday):
    first_day_exposition = wtday['first_day_exposition']
    days_exposition = wtday['days_exposition']
    if math.isnan(days_exposition):
        return (datetime.now() - first_day_exposition).days
    else:
        return days_exposition

In [ ]:
#Заполним пропуски
data['days_exposition'] = data.apply(days_exposition, axis=1)

In [ ]:
# Проверим все ли пропуски были убраны
data.info()

Все пропуски обработаны, пропусков больше нет. В процессе обработки в некоторых столбцах был изменён тип данных.


<a id="изменим-тип-данных">**Изменим тип данных в столбцах**</a>

Где это будет более логично и удобно для дальнейшей работы с данными

In [ ]:
# Изменим тип данных в столбце floors_total так как тут подразумевается целое число этажей

data['floors_total'] = data['floors_total'].astype('int')

In [ ]:
# Изменим тип данных в столбце days_exposition на целые числа, так как речь идёт о днях

data['days_exposition'] = data['days_exposition'].astype('int')

In [ ]:
# изменим тип данных в столбце balcony на целочисленный
data['balcony'] = data['balcony'].astype('int')

<a id="аномальные-значения-и-выбросы">**Аномальные значения и выбросы**</a>

Проверим весь датафрейм на аномальные значения с помощью метода .describe()

In [ ]:
data.describe()

Данные в этих столбцах содержат аномальные значения. Так как эти данные будут в дальнейшем учитываться в анализе, то аномальные значения удалим.

**total_area** - среднее не сильно отличается от медианы, минимальное значение допустимо, а вот максимальное значение и стандартное отклонение говорит о наличии аномальных значений.

**rooms** - максимальное значение говорит о наличии аномальных значений.

**ceiling_height** - максимальное и минимальные значения говорит о налии аномальных значений.

**floors_total** - стандартное отклонение и максимальное значение говорят о наличии аномальных значений

**living_area** - стандартное отклонение, минимальное и максимальные значения говорят об аномальных значениях

**kitchen_area** - стандартное отклонение и максимальное значение говорят о наличии аномальных значений

**last_price** - в максимальных и минимальных значениях видно, что есть аномальные значения

In [ ]:
# Построим гистограмму распредения цен в столбце last_price

data['last_price'].hist(bins=300)

В столбце last_price есть аномально маленькое значение и анамально большое. На гистограмме видим, что оосновная часть данных до 50 млн рублей. Отсечём данные больше 50 млн. рублей. И посмотрим сколько квартир до 1 000 000 руб. и сделаем отсечку минимальной цены.

In [ ]:
data['last_price'].hist(bins=300, figsize=(10, 3), range=(0,1000000))

In [ ]:
# Сделаем срез данных больше 600 тысяч и меньше 50 млн

data = data.query('last_price<=50000000 and last_price>600000')

In [ ]:
# Построим гистограмму, чтоб найти аномальные значения площади

data['total_area'].hist(bins=100, figsize=(10, 3))

На гистограмме видно, что до 300 квадратных метров квартиры есть и не одна. Но дальше от 300 - это по всей вероятности единичные объявления и вероятно с неверными значениями.

In [ ]:
# Сделаем срез по столбцу 'total_area', где оставим только значения меньше 300

data = data.query('total_area<=300')

In [ ]:
# Найдём аномалии в количестве комнат

data['rooms'].hist(bins=100, figsize=(10, 3))

Так видно что количество комнат заканчивается на 7 и дальше видимо идут редки значения, которыми можно пренебречь

In [ ]:
# Сделаем срез по столбцу 'rooms', где оставим только значения меньше или раыне 7

data = data.query('rooms<=7 and rooms>=0')

In [ ]:
data.query('rooms==0')

In [ ]:
# Найдём аномалии в "ceiling_height"
# Подберём диапозон значений, в котором будет лежать график, чтоб можно было рассмотреть значения

data['ceiling_height'].hist(bins=100, figsize=(10, 3), range=(2, 5))

Видим что значения лежат в  интервале от 2.5 до 4 метров. Сделаем такой срез, остальные удалим

In [ ]:
data = data.query('ceiling_height<=4 and ceiling_height>2.5')

In [ ]:
# Посмтрим на значения "floors_total"

data['floors_total'].hist(bins=100, figsize=(10, 3))

In [ ]:
# Сделаем срез данных от 1 до 30 этажей

data = data.query('floors_total<=30 and floors_total>=1')

In [ ]:
# Посмтрим на значения "living_area"

data['living_area'].hist(bins=100, figsize=(10, 3), range=(15, 170))

In [ ]:
# Сделаем срез данных от 10 до 160

data = data.query('living_area>=10 and living_area<=160')

In [ ]:
# Посмтрим на значения "kitchen_area"

data['kitchen_area'].hist(bins=100, figsize=(10, 3), range=(1, 40))

In [ ]:
#Сделаем срез данных от 4 до 40

data = data.query('kitchen_area>=4 and kitchen_area<=40')

Сравним исходный датасет и тот, который получили в результате фильтрации

In [ ]:
print('Исходный датасет:')
display(data_raw.describe())
print('Отфильтрованый датасет:')
display(data.describe())

**Промежуточные итоги:**

- Отфильтрованный датасет стал меньше на 4000 записей
- По столбцам минимальные, максимальные и средние значения имеют меньший разброс
- Удалили пустые строки, аномальные и нулевые значения

<a id="неявные-дубликаты">**Неявные дубликаты**</a>

В столбце "locality_name" - название населённого пункта, могут быть неявные неявные дубликаты. Где-то написано название и тип населённого пункта, где-то только название. Сделаем, чтобы были только названия.

In [ ]:
#Сбросим индексы, так как удаляли значения

data = data.reset_index(drop=True)

In [ ]:
# В цикле пройдём по значениям, разобъём каждое значение на слова и возьмём последнее в качестве названия

for index in range(len(data['locality_name'])):
    locality_name_str = data.loc[index, 'locality_name']
    data.loc[index, 'locality_name'] = locality_name_str.split()[-1]

In [ ]:
# Посмотрим как сработал алгоритм и какие остались значения

data['locality_name'].unique()

## <a id="расчёты">3. Расчёты и добавление результатов в таблицу</a>

В данном разделе проведём расчёты и добавим их в таблицу:
- Цена квадратного метра;
- День недели, месяц и год публикации объявления;
- Этаж квартиры; варианты — первый, последний, другой;
- Соотношение жилой и общей площади, а также отношение площади кухни к общей.

**Стоимость квадратного метра**

Рассчитаем по формуле "Стоимость квартиры" разделим на "Общую площадь"

In [ ]:
# Добавим столбец metr_pice - стоимость квадратного метра
# Рассчитывать стоимость квадратного метра

data['metr_price'] = data['last_price'] / data['total_area']

print(data['metr_price'].head(10))

Добавим столбец, в котором будет написан день недели публикации объявления

In [ ]:
# Добавим столбец week_day_exposition - день недели публикации

data['week_day_exposition'] = data['first_day_exposition'].dt.weekday

In [ ]:
# Сделаем функцию, которая на выходе возвращает день недели

def wd(day_number):
    if day_number == 0:
        return 'Понедельник'
    if day_number == 1:
        return 'Вторник'
    if day_number == 2:
        return 'Среда'
    if day_number == 3:
        return 'Четверг'
    if day_number == 4:
        return 'Пятница'
    if day_number == 5:
        return 'Суббота'
    if day_number == 6:
        return 'Воскресенье'

In [ ]:
# Применим функцию к столбцу week_day_exposition и заменим цифры значеним дня недели

data['week_day_exposition'] = data['week_day_exposition'].apply(wd)

In [ ]:
# Проверим как заполнился столбец

data['week_day_exposition'].value_counts()

Добавим столбец, в котором будет написан месяц публикации объявления

In [ ]:
# Добавим столбец month_exposition - месяц публикации

data['month_exposition'] = data['first_day_exposition'].dt.month

In [ ]:
# Сделаем функцию, которая на выходе возвращает месяц

def month(month_number):
    if month_number == 1:
        return 'Январь'
    if month_number == 2:
        return 'Февраль'
    if month_number == 3:
        return 'Март'
    if month_number == 4:
        return 'Апрель'
    if month_number == 5:
        return 'Май'
    if month_number == 6:
        return 'Июнь'
    if month_number == 7:
        return 'Июль'
    if month_number == 8:
        return 'Август'
    if month_number == 9:
        return 'Сентябрь'
    if month_number == 10:
        return 'Октябрь'
    if month_number == 11:
        return 'Ноябрь'
    if month_number == 12:
        return 'Декабрь'

In [ ]:
# Применим функцию к столбцу month_exposition и заменим цифры значеним

data['month_exposition'] = data['month_exposition'].apply(month)

In [ ]:
# Добавим столбец, где будет год публикации объявления

data['year_exposition'] = data['first_day_exposition'].dt.year

In [ ]:
# Проверим как заполнился столбец

print(data['month_exposition'].value_counts())
print(data['year_exposition'].value_counts())

Добавим столбец в котором будет написан этаж: первый, последний или другой

Назовём столбец floor_category

In [ ]:
# Напишем функцию, которая определяет этаж и возвращает одно из трёх значений

def floor_category(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['floors_total']:
        return 'последний'
    else:
        return'другой'

In [ ]:
# Создади столбец floor_category и применим функцию, чтоб его заполнить

data['floor_category'] = data.apply(floor_category, axis=1)

In [ ]:
# Проверим заполнение столбца

data['floor_category'].value_counts()

Переведём расстояние до центра города в километры и округлим до целых значений

In [ ]:
data['cityCenters_nearest_km'] = (data['cityCenters_nearest'] / 1000).round().astype('int')

## <a id="исследовательский-анализ-данных">4. Исследовательский анализ данных</a>

<a id="изучим-параметры-квартир">**Изучим параметры квартир**</a>: 
- общая площадь; 
- жилая площадь; 
- площадь кухни; цена объекта; 
- количество комнат; 
- высота потолков; 
- этаж квартиры; 
- тип этажа квартиры («первый», «последний», «другой»); 
- общее количество этажей в доме; 
- расстояние до центра города в метрах; 
- расстояние до ближайшего аэропорта; 
- расстояние до ближайшего парка; 
- день и месяц публикации объявления.

Построим гистограммы для кажого из параметров

In [ ]:
# Гистограмма для общей площади

data['total_area'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма для жилой площади

data['living_area'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма для площади кухни

data['kitchen_area'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма для цены квартиры

data['last_price'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма количества комнат

data['rooms'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма высоты потолков

data['ceiling_height'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма этажа квартиры

data['floor'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма типа этажа квартиры

data['floor_category'].hist()

In [ ]:
# Гистограмма общего количества этажей

data['floors_total'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма расстояния до центра города в метрах

data['cityCenters_nearest'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма расстояние до аэропорта

data['airports_nearest'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма расстояние до ближайшего парка

data['parks_nearest'].hist(bins=100, figsize=(10, 5))

In [ ]:
# Гистограмма день публикации объявления

data['week_day_exposition'].hist(figsize=(10, 5))

In [ ]:
# Гистограмма месяц публикации объявления

data['month_exposition'].hist(figsize=(10, 5))

<a id="выводы-по-гистограммам">**Выводы на основе построенных гистограмм**</a>

- В продаже больше всего квартир с площадью от 30 до 80 квадратных метров
- Площадь кухни в большинстве случаев от 6 до 13 квадратных метров
- График цены сильно смещён влево с большими пиками, что может говорить примерно об одинаковых цена на большую часть квартир
- Больше всего на рынке представленны однокомнатные квартиры
- Высота потолков в основном 2.65 м
- Основная часть квартир в продаже находится с 1 по 5 этажи
- Больше всего продаётся квартир в пятиэтажных домах
- Много квартир, у которых расстояние до центра 0, вероятно эти квартиры расположены в центр
- Меньше всего новых объявлений появляется в выходные, больше всего в чевтерг
- В феврале и марте в продаже появлется больше всего квартир

<a id="изучим-время-продажи">**Изучим время продажи квартир**</a>

Построим гистограмму по времени продажи

In [ ]:
data['days_exposition'].hist(bins=200, figsize=(10, 5))

In [ ]:
# Посчитаем среднее и медианное время продажи

print('Среднее время продажи квартиры составляет :', int(data['days_exposition'].mean()),'дней')
print('Медианное время продажи квартиры составляет :', int(data['days_exposition'].median()),'дней')

**Промежуточные выводы:**:
- В основном квартира продаётся за 124 дня
- Быстра продажа - это до 0 до 50 дней
- Даже спустя 1500 дней квартиры продаются

<a id="факторы-влияют-на-цену">**Какие факторы влияют на стоимость квартиры**</a>

Посмотрим зависит ли цена от общей площади, жилой площади, площади кухни, количества комнат, этажа, на котором расположена квартира (первый, последний, другой), даты размещения (день недели, месяц, год).

In [ ]:
# График зависимости цены от общей площади

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(x = data['total_area'], y = data['last_price'])

In [ ]:
# Зависимости цены от жилой площади

data.pivot_table(index='living_area', values='last_price').plot(figsize=(10, 5))
#Посмотрим коэфициент Пирсона
print('коэфициент Пирсона для жилой площади и цены', data['living_area'].corr(data['last_price']))

In [ ]:
# Зависимости цены от площади кухни

data.pivot_table(index='kitchen_area', values='last_price').plot(figsize=(10, 5))
#Посмотрим коэфициент Пирсона
print('коэфициент Пирсона для площади кухни и цены', data['kitchen_area'].corr(data['last_price']))

In [ ]:
# Зависимость цены от количества комнат

data.pivot_table(index='rooms', values='last_price').plot(figsize=(10, 5))
#Посмотрим коэфициент Пирсона
print('коэфициент Пирсона для количества комнат и цены', data['rooms'].corr(data['last_price']))

In [ ]:
# Зависимость цены от категории этажа

data.pivot_table(index='floor_category', values='last_price').plot(figsize=(10, 5))

In [ ]:
# Зависимость цены от дня недели размещения объявления

data.pivot_table(index='week_day_exposition', values='last_price').plot(figsize=(10, 5))

In [ ]:
# Зависимость цены от месяца размещения объявления

data.pivot_table(index='month_exposition', values='last_price').plot(figsize=(10, 5))

In [ ]:
# Зависимость цены от года размещения объявления

data.pivot_table(index='year_exposition', values='last_price').plot(figsize=(10, 5))

**Промежуточные выводы:**

- Коэффициент Пирсона положительный, что говорит о линейно зависимости цены от площади кухни, общей площади и количества комнат
- Цена квартиры, зависит от её площади и количества комнат
- Самая низкая цена у квартир на первом этаже
- Квартиры, размещенные в воскресенье с самой низкой ценой, а в четверг - с самой большой
- В октябре самые низкие цены на квартиры, в августе и декабре - самые высокие
- С 2014 года цены на квартиры падали, а с 2018 начался обратный рост цен

<a id="10-населённых-пунктов">**10 населённых пунктов с наибольшим числом объявлений**</a>

Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений.

In [ ]:
#Сделаем сводную таблицу, куда возьмём среднюю стоимость квадратного метра и количество объявлений
#Запишем эту таблицу в новую переменную locality_top10
locality_top10 = data.pivot_table(index='locality_name', values='metr_price', aggfunc=['mean', 'count'])

#Переименуем столбцы mean_metr_price - средняя цена квадратного метра, count - количество объявлений
locality_top10.columns = ['mean_metr_price', 'count']

#Отсортируем и возьмём первые 10 с самым большим количеством объявлений
locality_top10 = locality_top10.sort_values(by='count', ascending=False).head(10)
print(locality_top10)

#Найдём населённые пункты с самой низкой и самой высокой стоимостью
print()
print('Самая высокая стоимость квадратного метра:', locality_top10['mean_metr_price'].max())
print('Самая низкая стоимость квадратного метра:', locality_top10['mean_metr_price'].min())

<a id="центр-санкт-петербург">**Предложения квартир в центрально части Санкт-Петербурга**</a>

- Выделим квартиры в Санкт-Петербурге
- Вычислим среднюю цену киллометра до центра
- Посмотрим, как зависит цена на объект недвижимости от расстояния до центра

In [ ]:
#Сделаем срез по Санкт-Петербургу и сохраним в переменную spb
spb = data.loc[data['locality_name'] == 'Санкт-Петербург'].copy()

#Рассчитаем стоимость километра до центра
spb['price_km_center'] = spb['cityCenters_nearest_km'] / spb['last_price']

#Средняя стоимость за киллометр в центре
print('Средняя стоимость одного километра в центре Санкт-Петербурга:', spb['price_km_center'].mean())

#Зависимость цены от расстояния до центра
data.pivot_table(index='cityCenters_nearest_km', values='last_price').plot(figsize=(10, 5))

**Промежуточный вывод:**

Видим, что стоимость квартиры ближе к центру выше и уменьшается с удалением от центра. На расстоянии 40 и 55 км. наблюдаем пики с ценой. Вероятно в этих случая на цену квартиры повлияли другие, значимые параметры.

## <a id="общий-вывод">5. Общие выводы исследования</a>

- На стоимость квартиры влияет её расположение относительно центра города
- Цена квартиры напрямую зависит от общей площиди, площади кухни и количества комнат
- Самая низкая цена квартир на первом этаже
- Квартиры, размещенные в воскресенье с самой низкой ценой, а в четверг - с самой большой
- Больше всего на рынке однокомнатных квартир
- В октябре самые низкие цены на квартиры, в августе и декабре - самые высокие
- Сейчас цены на квартиры ниже чем в 2014 году, но наметился тренд на увеличение цены с 2018 года
- В феврале и марте на рынке появляется больше всего предложиний
- В основном квартира продаётся за 124 дня. Быстра продажа - до 0 до 50 дней, но даже спустя 4 года квартиры продаются
- Были обнаружены пропуски, которые вероятно появились из-за недочётов в формах, которые заполняет пользователь